<a href="https://colab.research.google.com/github/gvanathip/webscraping/blob/main/CarparkBKK2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install requests

In [2]:
pip install beautifulsoup4

In [3]:
from bs4 import BeautifulSoup
import requests

In [22]:
urls = []
urls.append('https://thailand-parking.com/th/carpark/')
for i in range(95):
  if i > 1:
    url = 'https://thailand-parking.com/th/carpark/page/'+str(i)+'/'
    urls.append(url)

In [58]:
urls[0]

'https://thailand-parking.com/th/carpark/'

In [44]:
links = []
for url in urls:
  res = requests.get(url)
  res.encoding = "utf-8"
  soup = BeautifulSoup(res.text, 'html.parser')
  tag = soup.find_all('a')

  for i in tag:
    if 'carpark' in i.get('href'):
      if len(i.get('href')) > 40:
        if 'page' not in i.get('href'):
          links.append(i.get('href'))

In [46]:
links = list(set(links))

In [171]:
th_links = []
for i in links:
  th_links.append(i.replace('https://thailand-parking.com/carpark/','https://thailand-parking.com/th/carpark/'))

In [299]:
total = []
for th_link in th_links:
  res = requests.get(th_link)
  res.encoding = "utf-8"
  soup = BeautifulSoup(res.text, 'html.parser')

  taga = soup.find_all('a')

  latlng = []
  for i in taga:
    if 'data' in str(i):
      latlng.append(str(i).split(' ')[2].split('=')[1].replace('"',''))

  tagp = soup.find_all('p')

  contents = []
  for i in tagp:
    if 'class' not in str(i):
      contents.append(str(i).replace("<p>",'').replace('</p>',''))

  details = dict()
  fee = []
  for i in contents[1:]:
    if " : " in i:
      details[i.replace('\t','').replace('\n','').split(' : ')[0]] = i.replace('\t','').replace('\n','').split(' : ')[1]
    else:
      fee.append(i)
    
  details['address'] = contents[0]
  details['name'] = soup.find('h3').text
  details['lat'] = latlng[0].split(',')[0]
  details['lng'] = latlng[0].split(',')[1]

  fee_title = []
  for i in tagp:
    if 'mod-parking-single-right-icon' in str(i):
      fee_title.append(str(i.contents))

  fee_title_ = []
  for i in fee_title[1:]:
    fee_title_.append(i.split(",")[2].replace('\\t','').replace('\\n','').replace(']','').replace(' ','').replace('"','').replace("'",''))

  fee = dict(zip(fee_title_,fee))
  details.update(fee)

  total.append(details)

In [303]:
import pandas as pd

In [304]:
df = pd.DataFrame(total)

In [305]:
df.head()

,ระยะทาง,สถานีใกล้เคียง,เบอร์โทร,ใกล้สถานที่สำคัญ,ช่วงเวลาในการจอด,ลักษณะที่จอดรถ,ขนาดที่จอดรถ,address,name,lat,lng,รายชั่วโมง,รายเดือน
0,0mจาก-,"MRT สุขุมวิท,BTS อโศก",094-989-7664,-,-,-,- ช่อง,-,NPD PARKING 690,13.73088512377590,100.57327750103200,-,NaN
1,0mจาก-,"MRT สุขุมวิท,BTS อโศก",094-989-7664,-,-,-,- ช่อง,-,NPD PARKING 499,13.73276585873540,100.5602069029040,-,NaN
2,0mจาก-,"MRT สุขุมวิท,BTS อโศก",094-989-7664,-,-,-,- ช่อง,-,NPD PARKING 601,13.72166701016930,100.58045936855000,-,NaN
3,0mจากBTS ช่องนนทรี,BTS ช่องนนทรี,094-989-7664,-,-,-,- ช่อง,-,NPD PARKING 361,13.723847326827100,100.53112064468700,-,NaN
4,0mจาก-,"MRT สุขุมวิท,BTS อโศก",094-989-7664,-,-,-,- ช่อง,-,NPD PARKING 653,13.735113019730400,100.56764738838900,-,NaN


In [306]:
pip install xlsxwriter

     |████████████████████████████████| 153kB 13.7MB/s 


In [308]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [309]:
out_path = "/content/drive/MyDrive/public_database/CarParkBKK2.xlsx"

writer = pd.ExcelWriter(out_path, engine='xlsxwriter')
df.to_excel(writer, index=False)
writer.save() 